##### Recommender System

Rating a movie from 1 to 5

Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from pathlib import Path
import os

Importing the dataset

In [2]:
cwd = Path.cwd()
path = cwd / 'datasets/Movie Recommender'
movies = pd.read_csv(os.path.join(path, 'ml-1m/movies.dat'), sep='::',
                     header = None, engine='python',
                     encoding='latin-1')

# movie_id | movie name | genre
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
users = pd.read_csv(os.path.join(path, 'ml-1m/users.dat'), sep='::',
                     header = None, engine='python',
                     encoding='latin-1')

# user_id | gender | age | code ~users job | visit code
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [4]:
ratings = pd.read_csv(os.path.join(path, 'ml-1m/ratings.dat'), sep='::',
                     header = None, engine='python',
                     encoding='latin-1')

# users | movie ids | ratings (1 - dislike, 5 - liked) | some timestamps
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


Preparing the training set and the test set

In [5]:
training_set = pd.read_csv(os.path.join(path, 'ml-100k/u1.base') , delimiter='\t')
training_set = np.array(training_set, dtype='int')

In [6]:
test_set = pd.read_csv(os.path.join(path, 'ml-100k/u1.test'), delimiter='\t')
test_set = np.array(test_set, dtype='int')

Getting the number of users and movies

In [7]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

Converting the data into an array with users in lines and movies in columns

In [8]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users+1):
    id_movies = data[:,1][data[:, 0] == id_users]
    id_ratings = data[:,2][data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies-1] = id_ratings

    # if id_users == 1: print(ratings)
    new_data.append(list(ratings))

  return new_data

training_set = convert(training_set)
test_set = convert(test_set)

Converting the data into Torch tensors

In [9]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

Creating the architecture of the Neural Network

In [10]:
# Stacked Auto Encoders
class SAE(nn.Module):
  def __init__(self,):
    super(SAE, self).__init__()
    self.fc1 = nn.Linear(nb_movies, 20) 
    self.fc2 = nn.Linear(20, 10)
    self.fc3 = nn.Linear(10, 20)
    self.fc4 = nn.Linear(20, nb_movies) 
    self.activation = nn.Sigmoid()
    
  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return x

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(),
                          lr = 0.01,
                          weight_decay = 0.5)

Training the SAE

In [11]:
nb_epoch = 200

for epoch in range(1, nb_epoch+1):
  train_loss = 0
  s = 0.

  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()

    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data * mean_corrector)
      s += 1.
      optimizer.step()

  print('epoch: ' + str(epoch) + ' loss:' + str(train_loss / s))

epoch: 1 loss:tensor(1.7718)
epoch: 2 loss:tensor(1.0967)
epoch: 3 loss:tensor(1.0534)
epoch: 4 loss:tensor(1.0386)
epoch: 5 loss:tensor(1.0307)
epoch: 6 loss:tensor(1.0266)
epoch: 7 loss:tensor(1.0238)
epoch: 8 loss:tensor(1.0222)
epoch: 9 loss:tensor(1.0206)
epoch: 10 loss:tensor(1.0198)
epoch: 11 loss:tensor(1.0189)
epoch: 12 loss:tensor(1.0185)
epoch: 13 loss:tensor(1.0178)
epoch: 14 loss:tensor(1.0177)
epoch: 15 loss:tensor(1.0173)
epoch: 16 loss:tensor(1.0170)
epoch: 17 loss:tensor(1.0169)
epoch: 18 loss:tensor(1.0167)
epoch: 19 loss:tensor(1.0163)
epoch: 20 loss:tensor(1.0161)
epoch: 21 loss:tensor(1.0161)
epoch: 22 loss:tensor(1.0160)
epoch: 23 loss:tensor(1.0159)
epoch: 24 loss:tensor(1.0158)
epoch: 25 loss:tensor(1.0159)
epoch: 26 loss:tensor(1.0156)
epoch: 27 loss:tensor(1.0155)
epoch: 28 loss:tensor(1.0150)
epoch: 29 loss:tensor(1.0125)
epoch: 30 loss:tensor(1.0109)
epoch: 31 loss:tensor(1.0092)
epoch: 32 loss:tensor(1.0067)
epoch: 33 loss:tensor(1.0070)
epoch: 34 loss:tens

Testing the SAE

In [12]:
test_loss = 0
s = 0.

for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)

  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data * mean_corrector)
    s += 1.

print('test loss:' + str(test_loss / s))

test loss:tensor(0.9500)
